# Plotting Emissions data from HERWIG Simulation
Rosie Schiffmann <br>
University of Manchester <br>
July 2025

# 1. Introduction
In this notebook, energy and power consumption data obtained from running particle physics event generation simulations using HERWIG is plotted. CPU and RAM energy and power consumption, alongside estimated CO2e emissions was tracked by CodeCarbon. The simulation complexity was varied across runs by changing the total number of events generated by HERWIG, and tracking was performed separately for the integration and generation phases of simulation. During the integration phase, the total cross section is calculated by numerically integrating the squared matrix elements over the allowed phase space of the process. In the generation phase, Herwig samples specific final states based on the probability distribution obtained in the integration phase, and then simulates parton showers, hadronisation, and hadron decays using Monte Carlo techniques.

Data for individual runs from CodeCarbon is stored in CSV files, that follow the naming convention YYYYMMDDType_metadata_EVENTS_JOBS.csv. YYYYMMDD represents the date that the simulations were run. Type is either Int or Gen, to represent data recorded duting the integration or generation phase respectively. EVENTS is the total number of simulated events. This variable can be changed inside the run_herwig_with_cc_loop.ipynb file. JOBS represnets the number of parallel jobs for HERWIG to use, in order to speed up computation by utilising multiple cores. Data that covers the emissions produced by the entire 10 runs as a whole can be found in CSV files YYYYMMDDType_emissions_EVENTS-JOBS.csv.

Plotting functions in this notebook are taken from the emission_tracking.ipynb notebook in https://github.com/rosieschiffmann/event-Transport-Simulation-Energy-Estimation github repository. 

In [2]:
#import python libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import re

# 2. Data Processing

Here, we will assimilate all of the individual CSV files into 2 separate master CSV files: "Int_master_emissions_data.csv" and "Gen_master_emissions_data.csv". To do this, mean values were calculated for emissions, duration and CPU/GPU energy and power consumption for each raw data file corresponding to a different number of events.

In [ ]:
#identify relevant csv files inside repository
int_files = glob.glob("*Int_metadata-*-4.csv")
gen_files = glob.glob("*Gen_metadata-*-4.csv")

def generate_master_csv(files, type):
	"""
	Function to generate a master csv file containing all data for plotting, for either the integration or generation
	phase of HERWIG.

	Parameters:
	- files (list of str): list of filenames to include in master csv
	- type (str): "Int" or "Gen" for integration phase data or generation phase data respectively
	"""
	summary_data = []

	for file in files:
		#identify and find mean of relevant data from CodeCarbon raw outputs
		df = pd.read_csv(file)
		mean_emissions = df['emissions'].mean()
		mean_duration = df['duration'].mean()
		mean_cpu_power = df['cpu_power'].mean()
		mean_ram_power = df['ram_power'].mean()
		mean_cpu_energy = df['cpu_energy'].mean()
		mean_ram_energy = df['ram_energy'].mean()
		#obtain number of events from filename
		match = re.search(r'_metadata-(\d+)-4\.csv', file)
		events = int(match.group(1)) if match else None

		#propagate errors
		emissions_err = df['emissions'].std() / np.sqrt(len(df['emissions']))
		duration_err = df['duration'].std() / np.sqrt(len(df['duration']))
		cpu_power_err = df['cpu_power'].std() / np.sqrt(len(df['cpu_power']))
		ram_power_err = df['ram_power'].std() / np.sqrt(len(df['ram_power']))
		cpu_energy_err = df['cpu_energy'].std() / np.sqrt(len(df['cpu_energy']))
		ram_energy_err = df['ram_energy'].std() / np.sqrt(len(df['ram_energy']))
		
		#add dictionary of data to summary_data list 
		summary_data.append({
			'filename': file,
			'number_of_events': events,
			'mean_emissions': mean_emissions,
			'emissions_err' : emissions_err,
			'mean_duration': mean_duration,
			'duration_err' : duration_err,
			'mean_cpu_power': mean_cpu_power,
			'cpu_power_err' : cpu_power_err,
			'mean_ram_power' : mean_ram_power,
			'ram_power_err' : ram_power_err,
			'mean_cpu_energy' : mean_cpu_energy,
			'cpu_energy_err' : cpu_energy_err,
			'mean_ram_energy' : mean_ram_energy,
			'ram_energy_err' : ram_energy_err,
			'cpu_energy_per_event' : mean_cpu_energy / events,
			'cpu_energy_per_event_err' : cpu_energy_err / events,
			'ram_energy_per_event' : mean_ram_energy / events,
			'ram_energy_per_event_err' : ram_energy_err / events,
			'duration_per_event' : mean_duration / events,
			'duration_per_event_err': duration_err / events
		})

	#convert into dataframe and create master csv file.
	summary_df = pd.DataFrame(summary_data)
	summary_df = summary_df.sort_values(by='number_of_events') #sort wrt number_of_events
	summary_df.to_csv(f"{type}_master_emissions_data.csv", index=False)
	return summary_df

int_summary_df = generate_master_csv(int_files, "Int")
gen_summary_df = generate_master_csv(gen_files, "Gen")


,filename,number_of_events,mean_emissions,emissions_err,mean_duration,duration_err,mean_cpu_power,cpu_power_err,mean_ram_power,ram_power_err,mean_cpu_energy,cpu_energy_err,mean_ram_energy,ram_energy_err,cpu_energy_per_event,cpu_energy_per_event_err,ram_energy_per_event,ram_energy_per_event_err,duration_per_event,duration_per_event_err
0,20250730Int_metadata-1000-4.csv,1000,0.000009,9.476505e-08,2.642078,0.027113,42.500000,0.000000,10.0,0.0,0.000031,3.228918e-07,0.000007,7.596954e-08,3.120056e-08,3.228918e-10,7.340078e-09,7.596954e-11,0.002642,0.000027
1,20250730Int_metadata-2000-4.csv,2000,0.000009,1.380562e-07,2.644371,0.039817,42.500000,0.000000,10.0,0.0,0.000031,4.704237e-07,0.000007,1.106477e-07,1.561228e-08,2.352119e-10,3.672656e-09,5.532384e-11,0.001322,0.000020
2,20250731Int_metadata-4000-4.csv,4000,0.000004,2.423705e-07,3.873004,0.245707,4.821200,0.002008,10.0,0.0,0.000005,3.321603e-07,0.000011,6.879689e-07,1.296320e-09,8.304008e-11,2.688243e-09,1.719922e-10,0.000968,0.000061
3,20250730Int_metadata-5000-4.csv,5000,0.000009,1.976548e-07,2.729403,0.056636,42.500000,0.000000,10.0,0.0,0.000032,6.735342e-07,0.000008,1.583848e-07,6.447421e-09,1.347068e-10,1.516709e-09,3.167695e-11,0.000546,0.000011
4,20250731Int_metadata-6000-4.csv,6000,0.000003,9.016555e-08,3.534810,0.092072,4.819263,0.001077,10.0,0.0,0.000005,1.238255e-07,0.000010,2.556788e-07,7.883376e-10,2.063759e-11,1.635508e-09,4.261314e-11,0.000589,0.000015
5,20250730Int_metadata-7000-4.csv,7000,0.000009,7.203724e-08,2.665577,0.020806,42.500000,0.000000,10.0,0.0,0.000031,2.454399e-07,0.000007,5.776124e-08,4.496700e-09,3.506284e-11,1.057828e-09,8.251606e-12,0.000381,0.000003
6,20250731Int_metadata-8000-4.csv,8000,0.000003,2.418887e-08,3.380669,0.025122,4.819245,0.000625,10.0,0.0,0.000005,3.324375e-08,0.000009,6.856891e-08,5.655047e-10,4.155469e-12,1.173208e-09,8.571114e-12,0.000423,0.000003
7,20250730Int_metadata-9000-4.csv,9000,0.000013,3.147656e-06,3.614495,0.911331,42.500000,0.000000,10.0,0.0,0.000043,1.072769e-05,0.000010,2.520634e-06,4.739512e-09,1.191965e-09,1.114584e-09,2.800704e-10,0.000402,0.000101
8,20250730Int_metadata-10000-4.csv,10000,0.000009,1.478222e-06,3.077951,0.105269,42.500000,0.000000,10.0,0.0,0.000036,1.242357e-06,0.000009,2.922625e-07,3.633368e-09,1.242357e-10,8.546851e-10,2.922625e-11,0.000308,0.000011
